In [41]:
import pdfplumber
import pandas as pd
import re
import os
import datetime

from etl.log import get_logger

logger = get_logger

def build_date(filename):
    global logger
    name_parts = filename.split("_")
    if len(name_parts) >= 4:
        sales_year = int(name_parts[1])
        sales_day = int(name_parts[2])
        sales_month = int(name_parts[3])

        if sales_month > 12:
            sales_day = int(name_parts[3])
            sales_month = int(name_parts[2])

        if sales_day > 31:
            sales_day = int(name_parts[2][0:2])
    else:
        logger.info("filename is not formatted correctly")
        return None

    return datetime.datetime(sales_year, sales_month, sales_day)


In [42]:
FOLDER = "data"
EXTENSION_PDFS = [".pdf"]
REGEX = r"([0-9]+)\s([0-9]{1})\s(.+)\s(Grocery No Tax|Lottery|Grocery Tax|Beer|Scratch Sales|Automotive|Candy \/ Gum|Cigarette Carton|Cigarette Pack|Moist Tabacco|Liquor|Wine)\s([0-9]+)\s([$0-9.]+)\s([$0-9.]+)\s([0-9.%]+)\s([0-9.%]+)"

In [43]:

for file in os.listdir(FOLDER):
  filename, ext = os.path.splitext(file)
  if ext in EXTENSION_PDFS:
    sales_date = build_date(filename)
    pdf = pdfplumber.open(os.path.join(FOLDER, file))
    data = []
    volumes = []
    start_parse = False
    for page in pdf.pages:
      rows = page.extract_text_lines()
      orig_record = {}
      for item in rows:
        if 'PLU No.' in item['text']:
          start_parse = True
        else:  
          if start_parse:
            matches = re.finditer(REGEX, item['text'], re.MULTILINE)
            mat_list = list(matches)
            if len(mat_list) > 0:
              for idx, match in enumerate(mat_list, start=1):
                values = list(match.groups())
                data.append({
                  "sales_date": sales_date,
                  "product_id": values[0],
                  "package_qty": int(values[1]),
                  "product_name": values[2],
                  "product_category": values[3],
                  "sales_qty": float(values[4]),
                  "product_price": float(values[5].replace("$","")),
                  "sales_price": float(values[6].replace("$","")),
                  "category_pct": round(float(values[7].replace("%",""))/100,5),
                  "day_pct": round(float(values[8].replace("%",""))/100,5)
                })

    print(len(data))
    raw_df = pd.DataFrame(data)
    raw_df.to_csv(os.path.join(FOLDER,filename+'.csv'),index=False)

293
